In [14]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [15]:
# Import tensorflow.
import tensorflow as tf

# PIL (Pillow) for working with images.
import PIL
from PIL import Image , ImageOps

# Used for globbing up directories.
import glob

# The Data Tools 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import the Keras tools.
from tensorflow import keras

# Network type.
from tensorflow.keras.models import Sequential

# Layer information.
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout #This right here is the convolutional NN

# For editing images. 
from tensorflow.keras.preprocessing.image import ImageDataGenerator #image processor that keras has built in
# from tensorflow.keras.utils import to_categorical

print("Tensorflow version:", tf.__version__)
print("Pandas version:", pd.__version__)
print("Keras version:", keras.__version__)
print("Pillow version:",PIL.__version__,"\n")

physical_devices = tf.config.list_physical_devices('GPU') 
print("GPU's:",physical_devices)
for gpu_instance in physical_devices: 
    tf.config.experimental.set_memory_growth(gpu_instance, True)

Tensorflow version: 2.1.0
Pandas version: 1.1.5
Keras version: 2.2.4-tf
Pillow version: 8.1.0 

GPU's: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [16]:
src_path_train = "../../../Collab/joe/proj2" #good data handling, always keep data in same place so can always find the path
# The src command initializes Keras’ ImageDataGenerator class

In [17]:
train_datagen = ImageDataGenerator( #This is creating a training data GENERATOR, stnd up object with our settings and tell keras that each image you see, apply these things to it.
    #Maybe you have keras look at 5 or 10 images that it will use to train, instead of hundreds; trying to design this so we get enough resolution to capture gesture
    #By changing the below values, you get different resolutuion, these numbers are a range   
    rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True, #be able to see it upside down
        fill_mode="nearest",
        validation_split=0.20) # 20% validation data set

test_datagen = ImageDataGenerator(rescale=1 / 255.0)
#This above is changing the images so we can try and train on them

In [13]:
batch_size = 8 #Batch size is going to limited by GPUs for our project. 
train_generator = train_datagen.flow_from_directory( #This is the generator; flow from directory means that it takes directly from directory, which allows for RAM savings.
    #Keras will only take what it needs, and data is already categorized (make sure you categorize them!)
    directory=src_path_train,
    target_size=(300, 300), #This need to match/compatibility with input shape of network (the first layer)
    color_mode="grayscale", #color images used here, but can also use grey scale
    batch_size=batch_size,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=42
)
#The thesis of this notebook was to see if a knot was tied properly. 

FileNotFoundError: [Errno 2] No such file or directory: '../../../Collab/joe/proj2'

In [ ]:
type(train_generator) # memory address of where this will be

In [ ]:
#below is the network used; focus on structure here
#The way CNN works, series of convolutional steps that created layers that work as a filter
model = Sequential()
model.add(Conv2D(16,kernel_size=(5,5),activation='relu',input_shape=(300, 300, 3))) #input shape is a 300 x 300 x 3 pixels x 16 images which is HUGE
# 16 is the number of images or kernels we want to use. so 16 5x5 filters (or matrices) will run over image
#Each of these will give different information and hopefully each one gives us a little more agency with the original image
model.add(MaxPooling2D(pool_size=(2, 2))) 
#This helps to make the size of data more tenable; it finds the largest values (differences), to make filters smaller to manage them
# we might be losing resolution when pool
#CONVOLUTION MAKES FILTERS, POOLING MAKES THEM SMALLER
model.add(Conv2D(16,kernel_size=(5,5),activation='relu')) # 5 x 5 kernel
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(100, activation='relu')) #push flattened layers into dense NN
model.add(Dropout(.5))
model.add(Dense(20, activation='relu'))
model.add(Dense(3, activation='softmax'))
          
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy']) #classified info comes out, ie what kind of knot is it?

In [ ]:
model.fit(train_generator, steps_per_epoch = train_generator.n//train_generator.batch_size, epochs=50)